# Laboratoire 0 - Exemple

Bienvenue dans ce laboratoire d'exemple. L'objectif ici est de vous familiariser avec l'environnement Jupyter, ainsi que les interactions que vous pouvez avoir avec le robot.

Jupyter nous sert d'interface pour appeler du code python. Débutons 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import time
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import animation
mpl.rc('animation', html='html5') #display animated plots inline

from websocket_client import Robot, Sensors

Modifiez la valeur de `ip_robot` selon le robot qui vous a été assigné.

In [ ]:
ip_robot = '10.248.0.32'
robot = Robot(ip_robot)

In [ ]:
robot.connect()

In [ ]:
robot.listen_to(Sensors.hokuyo)

In [ ]:
fig, ax = plt.subplots(1,1, subplot_kw=dict(projection='polar'))

ax.grid(True)
ax.set_rmax(3.0)
ax.set_rmin(0.0)
ax.set_theta_zero_location('N')

last_data = robot.read_sensor_data(Sensors.hokuyo)['msg']
ranges = np.array(last_data['ranges'])
angles = np.arange(last_data['angle_min'], last_data['angle_max'] + last_data['angle_increment'], 
                   last_data['angle_increment'])

ranges = ranges.astype('float32')


scatter_plot = ax.scatter(angles, ranges, color='b', s=2, animated=True)

plt.show()

In [ ]:
def init_frame():
    scatter_plot.set_array([])
    return (scatter_plot,)

def animate(i):
    last_data = robot.peek_most_recent_data(Sensors.hokuyo)['msg']
    ranges = np.array(last_data['ranges'])
    ranges = np.nan_to_num(ranges.astype('float32'))
    angles = np.arange(last_data['angle_min'], last_data['angle_max'] + last_data['angle_increment'], 
                       last_data['angle_increment'])
    
    table = np.zeros((ranges.shape[0], 2))
    table[:, 0] = angles
    table[:, 1] = ranges
    scatter_plot.set_offsets(table)
    
    return scatter_plot,

anim = animation.FuncAnimation(fig, animate,frames=100, interval=100, blit=True)

In [ ]:
anim